In [ ]:
import json
import pandas as pd

import argparse
import io
import os
from datetime import date
# Imports the Google News api
from newsapi import NewsApiClient

# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


In [ ]:
class create_df:
    def __init__(self, cur_original, cur_new, news_api_key, json_google_cloud):
        self.original = cur_original
        self.new = cur_new
        self.newsapi = NewsApiClient(api_key=news_api_key)
        self.client = language.LanguageServiceClient.from_service_account_json(json_google_cloud)
        
        
    # classify text into categories   
    def classify(self, text):
        client = self.client
        document = language.types.Document(content=text, type=language.enums.Document.Type.PLAIN_TEXT)
        response = client.classify_text(document)
        categories = response.categories

        result = {}

        for category in categories:
            categories = category.name.split("/")
            categories.pop(0)
            for i in range(len(categories)):
                result[categories[i]] = category.confidence

        return result
    
    
    # retrieve articles from google news api
    def retrieve_articles(self):
        today = date.today().strftime("%Y/%m/%d").replace("/","-")
        # grabbing articles
        newsapi = self.newsapi
        
        all_articles = newsapi.get_everything(q=self.original, from_param='2019-08-15', to=today, language='en', sort_by='relevancy')
        articles = all_articles['articles']
        
        formatted_articles = []

        # creating new article object
        for article in articles:
            info = str(article['title']) + " " + str(article['description']) + " " + str(article['content'])
            classifications = self.classify(info)
            if classifications:
                individual_article = {}
                # just the date
                # print(article['publishedAt'][:10])
                individual_article['date'] = article['publishedAt']

                info = info.replace("'","") 
                individual_article['text'] = info

                formatted_articles.append(individual_article)
                
        self.formatted_articles = formatted_articles
#         print(json.dumps(self.formatted_articles, indent=4, sort_keys=True))
        return
    
    
    # Sentiment analysis using google cloud
    def sentiment_analysis(self):
        client = self.client
        tone_analyzed_articles = []
        formatted_articles = self.formatted_articles.copy()

        # The text to analyze
        for i in range(len(formatted_articles)):
            analyzed_article = {}
            article = formatted_articles[i]
            text = article['text']

            document = types.Document(content = text, type = enums.Document.Type.PLAIN_TEXT)

            # Detects the sentiment of the text
            sentiment = client.analyze_sentiment(document=document).document_sentiment

            analyzed_article['date'] = article['date']
            analyzed_article['score'] = sentiment.score
            analyzed_article['magnitude'] = sentiment.magnitude

            tone_analyzed_articles.append(analyzed_article)

        print(json.dumps(tone_analyzed_articles, indent=4, sort_keys=True))

        #     print('Text: {}'.format(text))
        #     print('Sentiment: {}, {}'.format(, sentiment.magnitude))
        
        
    def df_append(self):
        self.retrieve_articles()
        self.sentiment_analysis()

## Input Parameter

In [ ]:
news_api_key = 'a93fd9a824b848bca3436e761c5a0d4d'
json_google_cloud = 'htn2019-f059d4b46f7d.json'

In [ ]:
print("Enter the original currency: ")
# cur_original = input()
print("Enter the new currency: ")
# cur_return = input()

In [ ]:
cur_original = 'CAD'
cur_return = 'USD'

## Create Dataframe

In [ ]:
df = create_df(cur_original, cur_return, news_api_key, json_google_cloud)
df.df_append()